In [3]:
import sys

sys.path.append(r'C:\Users\User\Desktop\UAB\3rd-year\2nd-semester\synthesis project II\synthesis-project-II')

In [7]:
from Pol.AEpproach.src.modelPCA import Autoencoder
import torch
import dgl
from dgl.nn import GraphConv
from Paula.GCN import GCNModel
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import torch.nn as nn


In [ ]:
"""
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(166, 128),
            nn.LeakyReLU(),
            nn.Linear(128, 86),
            nn.LeakyReLU(),
            nn.Linear(86, 48),
            nn.LeakyReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(48, 86),
            nn.LeakyReLU(),
            nn.Linear(86, 128),
            nn.LeakyReLU(),
            nn.Linear(128, 166),
            nn.LeakyReLU() 
        )
    
    def forward(self, in_features):
        encoded_features = self.encoder(in_features)
        out = self.decoder(encoded_features)
        return out
"""

In [ ]:
"""
class GCNModel(torch.nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCNModel, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats, allow_zero_in_degree=True)
        self.conv2 = GraphConv(h_feats, num_classes, allow_zero_in_degree=True)

    def forward(self, g, inputs):
        h = self.conv1(g, inputs)
        h = torch.relu(h)
        h = self.conv2(g, h)
        return h
"""

In [5]:


#load datasets
df_classes = pd.read_csv(r"C:\Users\User\Desktop\UAB\3rd-year\2nd-semester\synthesis project II\elliptic_bitcoin_dataset\elliptic_txs_classes.csv")
df_edges = pd.read_csv(r"C:\Users\User\Desktop\UAB\3rd-year\2nd-semester\synthesis project II\elliptic_bitcoin_dataset\elliptic_txs_edgelist.csv")
df_features = pd.read_csv(r"C:\Users\User\Desktop\UAB\3rd-year\2nd-semester\synthesis project II\elliptic_bitcoin_dataset\elliptic_txs_features.csv", header=None)

df_classes = df_classes[df_classes['class'] != "unknown"]
# Change column names --> Column 1 is txId, Column 2 is timestep and the rest are unknown features
df_features.columns = ['txId', 'timestep'] + ['f' + str(i) for i in range(165)]

# Remove all edges that do not appear in classesDF
df_features = df_features[df_features['txId'].isin(df_classes['txId'])]

df_edges = df_edges[df_edges['txId1'].isin(df_classes['txId']) & df_edges['txId2'].isin(df_classes['txId'])]

# Identifica los nodos en df_classes y df_features
nodos_classes = set(df_classes['txId'])
nodos_features = set(df_features['txId'])

# Encuentra la intersección de nodos entre clases y características
nodos_comunes = nodos_classes.intersection(nodos_features)

# Filtrar df_edges para asegurar que ambos nodos de cada arista estén en nodos_comunes
df_edges = df_edges[df_edges['txId1'].isin(nodos_comunes) & df_edges['txId2'].isin(nodos_comunes)]

# Filtrar df_features y df_classes para incluir solo los nodos comunes
df_features = df_features[df_features['txId'].isin(nodos_comunes)]
df_classes = df_classes[df_classes['txId'].isin(nodos_comunes)]

# Asegurarte de que df_edges solo contiene nodos presentes en los otros DataFrames
assert all(df_edges['txId1'].isin(df_features['txId']))
assert all(df_edges['txId2'].isin(df_features['txId']))

# Verificar que df_features y df_classes contienen los mismos nodos
assert set(df_features['txId']) == set(df_classes['txId'])

#We assign 0 to ilicit transacions and 1 to licit ones
df_classes['class'] = df_classes['class'].replace({'1': 0, '2': 1})

unique_nodes_classes = df_classes['txId'].nunique()

unique_nodes_edges = pd.concat([df_edges['txId1'], df_edges['txId2']]).nunique()

unique_nodes_features = df_features['txId'].nunique()

#reindexar los nodos para que empicen desde 0
node_mapping = {old_id: new_id for new_id, old_id in enumerate(df_features['txId'].unique())}

# Aplicar este mapeo a df_edges para actualizar los identificadores de nodos a la nueva secuencia
df_edges['txId1'] = df_edges['txId1'].map(node_mapping)
df_edges['txId2'] = df_edges['txId2'].map(node_mapping)

# Ahora, cuando crees el grafo y asignes características y etiquetas, el tamaño debería coincidir
g = dgl.graph((df_edges['txId1'].values, df_edges['txId2'].values))
g.ndata['feat'] = torch.tensor(df_features.iloc[:, 2:].values, dtype=torch.float32)  # Asumiendo que las dos primeras columnas no son características
g.ndata['label'] = torch.tensor(df_classes['class'].astype(int).values, dtype=torch.long)  # Asegurándonos de que las clases están en formato numérico



In [8]:
#cargar todos los modelos

#autoencoder
model_AE = Autoencoder()
model_AE.load_state_dict(torch.load(r'C:\Users\User\Desktop\UAB\3rd-year\2nd-semester\synthesis project II\synthesis-project-II\Pol\AEpproach\models\Autoencoder_secondRUN.pth', map_location=torch.device('cpu')))

#random forest
randomforest = RandomForestClassifier(n_estimators=100, random_state=42)

#contrastive
#contrastive = 

#GCN

GCN = GCNModel(g.ndata['feat'].shape[1], 16, 2) 
GCN.load_state_dict(torch.load(r'C:\Users\User\Desktop\UAB\3rd-year\2nd-semester\synthesis project II\synthesis-project-II\Paula\best_model_GCN.pth'))

<All keys matched successfully>